In [7]:
from collections import Counter
from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit import Parameter
import numpy as np
from itertools import combinations, chain


def append_multi_rzz_term(qc, qubits, gamma):
    """实现多体 Z 相互作用项 e^{-i \gamma Z_1 Z_2 \dots Z_k}"""
    if len(qubits) == 0:
        return
    if len(qubits) == 1:
        qc.rz(2 * gamma, qubits[0])
        return
    target = qubits[-1]
    control_qubits = qubits[:-1]
    for control in control_qubits:
        qc.cx(control, target)
    qc.rz(2 * gamma, target)
    for control in reversed(control_qubits):
        qc.cx(control, target)


def append__zz_term(qc, qubit1, qubit2, gamma):
    qc.cx(qubit1, qubit2)
    qc.rz(2 * gamma, qubit2)
    qc.cx(qubit1, qubit2)


def get_cost_circuit_FOOSP(gamma, qc, n, m, l, N, p_ij, W, b):
    """构建成本哈密顿量 H_C 的量子线路"""
    # 第一部分：C_max 部分，-1/2 sum_h 2^h Z_zh
    for h in range(l):
        coeff = -0.5 * 2 ** h
        qc.rz(-2 * gamma * coeff, n * N + h)  # RZ(-gamma * 2^h, z_h)

    # 第二部分：不可用机器惩罚 K_1 sum_{j in W} sum_i f_binary_j(s_i)
    K1 = 100
    for j in W:
        for i in range(n):
            # 生成所有子集 S，包括空集
            all_subsets = list(chain.from_iterable(combinations(range(N), r) for r in range(N + 1)))
            for S in all_subsets:
                if not S:  # 空集，系数为 1/2^N
                    coeff_S = K1 / (2 ** N)
                    qc.rz(2 * gamma * coeff_S, i * N)  # 任意一个比特应用相位，实际为全局相位
                else:
                    coeff_S = K1 / (2 ** N)
                    for k in S:
                        coeff_S *= (1 - 2 * b[j][k])
                    if coeff_S != 0:
                        qubits = [i * N + k for k in S]
                        append_multi_rzz_term(qc, qubits, gamma * coeff_S)

    # 第三部分：负载平衡惩罚 P sum_j [(load_j)^2 - 2 load_j C_max + C_max^2 / 4]
    P = 200
    for j in range(m):
        # (load_j)^2 部分
        for i in range(n):
            for i_prime in range(n):
                all_subsets_i = list(chain.from_iterable(combinations(range(N), r) for r in range(N + 1)))
                all_subsets_iprime = list(chain.from_iterable(combinations(range(N), r) for r in range(N + 1)))
                for S_i in all_subsets_i:
                    for S_iprime in all_subsets_iprime:
                        coeff_Si = 1
                        for k in S_i:
                            coeff_Si *= (1 - 2 * b[j][k])
                        coeff_Siprime = 1
                        for k in S_iprime:
                            coeff_Siprime *= (1 - 2 * b[j][k])
                        coeff = P * p_ij[i][j] * p_ij[i_prime][j] / (2 ** (2 * N)) * coeff_Si * coeff_Siprime
                        if coeff != 0:
                            qubits = [i * N + k for k in S_i] + [i_prime * N + k for k in S_iprime]
                            count = Counter(qubits)
                            qubits_final = [x for x in count if count[x] % 2 != 0]
                            if qubits_final:
                                append_multi_rzz_term(qc, qubits_final, gamma * coeff)

        # -2 load_j C_max 部分
        for i in range(n):
            all_subsets_i = list(chain.from_iterable(combinations(range(N), r) for r in range(N + 1)))
            for S_i in all_subsets_i:
                coeff_Si = 1
                for k in S_i:
                    coeff_Si *= (1 - 2 * b[j][k])
                if coeff_Si != 0:
                    coeff = -2 * P * p_ij[i][j] / (2 ** N) * coeff_Si * (2 ** l - 1)  # C_max 的常数项
                    qubits_load = [i * N + k for k in S_i]
                    append_multi_rzz_term(qc, qubits_load, gamma * coeff)
                    for h in range(l):
                        coeff_h = -2 * P * p_ij[i][j] / (2 ** N) * coeff_Si * 2 ** h
                        qubits_h = qubits_load + [n * N + h]
                        append_multi_rzz_term(qc, qubits_h, gamma * coeff_h)

        # C_max^2 / 4 部分
        for h in range(l):
            for h_prime in range(h, l):  # 避免重复
                if h == h_prime:
                    coeff = P / 4 * (2 ** h) * (2 ** h)  # Z_h^2 = I
                else:
                    coeff = P / 4 * (2 ** h) * (2 ** h_prime)
                    append__zz_term(qc, n * N + h, n * N + h_prime, 2 * gamma * coeff)


def get_mixer_circuit(beta, qc):
    """构建混合哈密顿量 H_B = sum X_i"""
    for i in range(qc.num_qubits):
        qc.rx(-2 * beta, i)


def build_qaoa_circuit(n, m, l, p_ij, W, b, p=3):
    """构建完整的 QAOA 线路"""
    N = int(np.ceil(np.log2(m)))
    num_qubits = n * N + l

    # 初始化电路
    qc = QuantumCircuit(num_qubits)
    qc.h(range(num_qubits))

    # 定义参数
    gamma = [0.1, 0.2, 0.3]
    beta = [0.1, 0.2, 0.3]

    # 应用 p 层 QAOA
    for i in range(p):
        get_cost_circuit_FOOSP(gamma[i], qc, n, m, l, N, p_ij, W, b)
        get_mixer_circuit(beta[i], qc)

    # 测量
    qc.measure_all()

    return qc


def main():
    # 示例参数
    n = 2
    m = 3
    N = int(np.ceil(np.log2(m)))
    l = 3
    p_ij = np.array([[1, 2, 3], [4, 5, 6]])  # 示例处理时间
    W = [2]  # 不可用机器索引
    b = []  # 机器二进制表示
    for j in range(m):
        binary_j = format(j, '0{}b'.format(N))
        b_jk = [int(bit) for bit in binary_j]
        b.append(b_jk)

    # 构建 QAOA 线路
    qc = build_qaoa_circuit(n, m, l, p_ij, W, b, p=3)

    # 模拟运行（示例）
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend, shots=10000)
    result = job.result()
    counts = result.get_counts()
    print("Measurement counts:", counts)

    return qc


if __name__ == "__main__":
    qc = main()
    print(qc)

C:\Users\李华宪\AppData\Local\Temp\ipykernel_45940\506101212.py:147: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  job = execute(qc, backend, shots=10000)


Measurement counts: {'1001100': 279, '1110110': 159, '0001110': 364, '0100101': 122, '0111011': 64, '1100100': 149, '1010101': 109, '1001000': 66, '1101111': 324, '0101100': 66, '0110100': 13, '0100001': 62, '0111111': 75, '1110010': 124, '0111100': 69, '0100100': 122, '0001101': 135, '0111010': 38, '0100110': 95, '0001111': 94, '1110101': 126, '0100000': 55, '1001111': 13, '1110011': 28, '1011110': 316, '1110100': 45, '1001010': 53, '0010111': 44, '1101011': 80, '0101010': 30, '0110110': 93, '0000001': 109, '1000111': 62, '1111011': 93, '0110001': 131, '0101111': 129, '1000000': 9, '1111110': 44, '0000110': 273, '0000101': 87, '1000011': 36, '1111111': 389, '0001100': 103, '1001110': 7, '1110000': 138, '0011101': 123, '1010010': 121, '1111101': 126, '1000001': 138, '0000111': 90, '0101101': 69, '0110011': 27, '0000100': 56, '1000010': 203, '1101100': 18, '0010110': 216, '0010000': 82, '1101110': 50, '1010001': 75, '0010001': 35, '0011111': 76, '1100011': 76, '1011101': 325, '1111010':